In [65]:
import numpy as np
import tensorflow as tf
import random

# NLTK one_hot encoding test

In [66]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()


# import intents file
import json
with open('dataset.json') as json_data:
    intents = json.load(json_data)

In [67]:
words = []
classes = []
documents = []
ignore_words = ['?']
# loop through each sentence in our intents utterances
for intent in intents['intents']:
    for pattern in intent['utterances']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['intent']))
        # add to our classes list
        if intent['intent'] not in classes:
            classes.append(intent['intent'])

# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))

print (len(classes), "classes", classes)
print (len(words), "unique stemmed words")
print (len(documents), "documents")
print (documents[0])
print (documents[1])




(25, 'classes', [u'about_VA', u'capabilities', u'compound_questions', u'decision_replies', u'goodbyes', u'greetings', u'improving_system', u'information_request', u'interface_interactions', u'interface_issues', u'locate_amenity', u'navigation', u'negative_reaction', u'not_specified', u'out_of_scope', u'phone', u'positive_reaction', u'selections', u'system_reliance', u'traffic_update', u'turn_down', u'turn_off', u'turn_on', u'turn_up', u'weather'])
(1320, 'unique stemmed words')
(2520, 'documents')
([u'hey', u',', u'now', u'it', u"'s", u'raining', u',', u'you', u'said', u'you', u'did', u"n't", u'know', u'anything', u'about', u'that', u'.', u'Are', u'you', u'a', u'liar', u'?'], u'system_reliance')
([u'do', u'you', u'tell', u'the', u'truth'], u'system_reliance')


In [68]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each intent and '1' for current intent
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

print (len(bag), "bags")    
print (len(training), "trainings")    

    
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])

(1320, 'bags')
(2520, 'trainings')


# Prepare Keras / TensorFlow model

In [69]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam


In [70]:
model = Sequential()
model.add(Dense(8, input_dim=len(train_x[0]), activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(len(train_y[0]), activation='softmax'))
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 8)                 10568     
_________________________________________________________________
dense_8 (Dense)              (None, 8)                 72        
_________________________________________________________________
dense_9 (Dense)              (None, 25)                225       
Total params: 10,865
Trainable params: 10,865
Non-trainable params: 0
_________________________________________________________________


In [71]:
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


# Train Model

In [72]:
model.fit(np.array(train_x), np.array(train_y), epochs=100)

Epoch 1/100
2520/2520 [==============================] - 0s 117us/step - loss: 3.0823 - acc: 0.2940
Epoch 2/100
2520/2520 [==============================] - 0s 48us/step - loss: 2.4973 - acc: 0.3119
Epoch 3/100
2520/2520 [==============================] - 0s 49us/step - loss: 1.9712 - acc: 0.4639
Epoch 4/100
2520/2520 [==============================] - 0s 48us/step - loss: 1.6765 - acc: 0.5603
Epoch 5/100
2520/2520 [==============================] - 0s 46us/step - loss: 1.4293 - acc: 0.6024
Epoch 6/100
2520/2520 [==============================] - 0s 48us/step - loss: 1.2346 - acc: 0.6548
Epoch 7/100
2520/2520 [==============================] - 0s 48us/step - loss: 1.1044 - acc: 0.6937
Epoch 8/100
2520/2520 [==============================] - 0s 48us/step - loss: 1.0099 - acc: 0.7222
Epoch 9/100
2520/2520 [==============================] - 0s 46us/step - loss: 0.9370 - acc: 0.7417
Epoch 10/100
2520/2520 [==============================] - 0s 51us/step - loss: 0.8759 - acc: 0.7738
Epoch 11

2520/2520 [==============================] - 0s 56us/step - loss: 0.0454 - acc: 0.9893
Epoch 84/100
2520/2520 [==============================] - 0s 53us/step - loss: 0.0442 - acc: 0.9901
Epoch 85/100
2520/2520 [==============================] - 0s 50us/step - loss: 0.0428 - acc: 0.9897
Epoch 86/100
2520/2520 [==============================] - 0s 55us/step - loss: 0.0413 - acc: 0.9913
Epoch 87/100
2520/2520 [==============================] - 0s 52us/step - loss: 0.0402 - acc: 0.9901
Epoch 88/100
2520/2520 [==============================] - 0s 58us/step - loss: 0.0391 - acc: 0.9905
Epoch 89/100
2520/2520 [==============================] - 0s 59us/step - loss: 0.0389 - acc: 0.9909
Epoch 90/100
2520/2520 [==============================] - 0s 53us/step - loss: 0.0372 - acc: 0.9893
Epoch 91/100
2520/2520 [==============================] - 0s 55us/step - loss: 0.0360 - acc: 0.9909
Epoch 92/100
2520/2520 [==============================] - 0s 53us/step - loss: 0.0355 - acc: 0.9905
Epoch 93/100


# Test Model

In [73]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [74]:
p = bow("Turn on the radio", words)
print (p)
print (classes)

[0 0 0 ... 0 0 0]
[u'about_VA', u'capabilities', u'compound_questions', u'decision_replies', u'goodbyes', u'greetings', u'improving_system', u'information_request', u'interface_interactions', u'interface_issues', u'locate_amenity', u'navigation', u'negative_reaction', u'not_specified', u'out_of_scope', u'phone', u'positive_reaction', u'selections', u'system_reliance', u'traffic_update', u'turn_down', u'turn_off', u'turn_on', u'turn_up', u'weather']


In [75]:
y_pred = model.predict(np.array([p]))

In [76]:
print(y_pred)

[[1.84779242e-15 1.48731374e-11 5.13623527e-04 4.74194820e-07
  4.60809862e-12 9.44413898e-08 4.74447486e-23 1.58046010e-09
  1.75314049e-17 3.78078738e-07 1.43997797e-13 1.91251182e-11
  4.92094938e-14 3.73306875e-16 8.15088157e-08 5.76928752e-16
  5.85259236e-15 3.10337365e-21 1.00107344e-11 1.04443245e-14
  2.23725988e-03 1.20815820e-08 9.95596945e-01 1.65118114e-03
  2.15891249e-15]]


In [77]:
print(y_pred[0][22])

0.99559695


In [78]:
print(classes[22])

turn_on


# Export Model using CoreML Tools

In [79]:
model.save('Car.h5')

In [80]:
import coremltools
coreml_model = coremltools.converters.keras.convert(model, input_names="utterances", output_names="entities")
coreml_model

0 : dense_7_input, <keras.engine.topology.InputLayer object at 0x12552a990>
1 : dense_7, <keras.layers.core.Dense object at 0x12552aa90>
2 : dense_7__activation__, <keras.layers.core.Activation object at 0x127f5f790>
3 : dense_8, <keras.layers.core.Dense object at 0x12552a550>
4 : dense_8__activation__, <keras.layers.core.Activation object at 0x127f5f810>
5 : dense_9, <keras.layers.core.Dense object at 0x12552acd0>
6 : dense_9__activation__, <keras.layers.core.Activation object at 0x127f5f890>


input {
  name: "utterances"
  type {
    multiArrayType {
      shape: 1320
      dataType: DOUBLE
    }
  }
}
output {
  name: "entities"
  type {
    multiArrayType {
      shape: 25
      dataType: DOUBLE
    }
  }
}

In [81]:
coreml_model.save('Car.mlmodel')